In [60]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
from pyemd import emd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [61]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

def combine(state, goal):
    if state is None:
        return None
    return torch.cat((state, goal), 1)

In [62]:
class miniDQN(nn.Module):

    def __init__(self, hidden):
        super(miniDQN, self).__init__()
        self.W1 = nn.Linear(8, hidden)
        self.W2 = nn.Linear(hidden, 4)
        nn.init.xavier_uniform_(self.W1.weight)
        nn.init.xavier_uniform_(self.W2.weight)

    def forward(self, x):
        x = F.relu(self.W1(x))
        return self.W2(x)

In [63]:
num_row = 5
num_col = 5

action_direction = [(0,1), (0,-1), (1,0), (-1,0)]
block = { "empty" : 0, "blue" : 1, "red" : 2, "goal" : 3, "wall" : 4}

reds = [(3,0)]
blues = [(0,3)]
start = (0,0)
end = (4,4)
walls = []

start_state = np.asarray(list(start) + [0,0])
end_state = np.asarray(list(end) + [1,1])


def is_done(s):
    row, col, blue, red = tuple(s)
    return blue and red and (row, col) == end

def step(s, a):
    row, col, blue, red = tuple(s)
    a_row, a_col = action_direction[a]

    row = row + a_row
    col = col + a_col

    if row < 0 or row >= num_row or col < 0 or col >= num_col or (row, col) in walls:
        return s

    blue = blue or (row, col) in blues
    red = red or (row, col) in reds

    return np.asarray((row, col, blue, red))

def get_reward(s, a, s_prime):
    if is_done(s):
        return 0.0
    return -1.0

def L1(s1, s2):
    return 1.0 * np.sum(np.abs(np.asarray(s1) - np.asarray(s2)))

In [64]:
#Very adhoc, requires no dynamics
distance_cache = {}

def distance(a, b):
    global distance_cache
    ta = tuple(a)
    tb = tuple(b)
    if (ta,tb) in distance_cache:
        return distance_cache[(ta,tb)]
    
    row_a, col_a, blue_a, red_a = ta
    row_b, col_b, blue_b, red_b = tb
    
    blue_match = blue_a == blue_b
    red_match = red_a == red_b
    
    if red_match and not blue_match:
        dist = min([L1((row_a,col_a),blue) + L1(blue, (row_b, col_b)) for blue in blues])
    elif not red_match and blue_match:
        dist = min([L1((row_a,col_a),red) + L1(red, (row_b, col_b)) for red in reds])
    elif not red_match and not blue_match:
        x = min([L1((row_a,col_a),red) + L1(red, blue) + L1(blue, (row_b, col_b)) for red, blue in zip(reds, blues)])
        y = min([L1((row_a,col_a),blue) + L1(blue, red) + L1(red, (row_b, col_b)) for red, blue in zip(reds, blues)])
        dist = min(x,y)
    else:
        dist = L1(ta, tb)
    distance_cache[(ta,tb)] = dist
    return dist

In [65]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 4000
TARGET_UPDATE = 50
HIDDEN_LAYER_SIZE = 20
EPISODE_LENGTH = 25
LEARNING_RATE = 0.0008

DIVERSE_EXPERT = True
BACKPLAY = True

num_episodes = 20000
random.seed(5)
distance_cache = {}

policy_net = miniDQN(HIDDEN_LAYER_SIZE).to(device)
target_net = miniDQN(HIDDEN_LAYER_SIZE).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr = LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.99999)
memory = ReplayMemory(40000)

In [86]:
steps_done = 0

def select_action(state, goal, test = False):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if test or sample > eps_threshold:
        with torch.no_grad():
            x = torch.cat((state, goal), 1)
            return policy_net(x).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(4)]], device=device, dtype=torch.long)

def select_action_entropy(state, goal, alpha):
    x = torch.cat((state, goal), 1)
    potential = (alpha * policy_net(x).detach()).exp().squeeze()
    dist = (potential / potential.sum()).numpy()
    return torch.tensor([[np.random.choice(4, p = dist)]], device=device, dtype=torch.long)
    

def select_expert_action(state, goal):
    row, col, blue, red = tuple(state)
    action_scores = np.zeros(4)
    for a in range(4):
        #Assumes determinism
        next_state = step(state, a)
        action_scores[a] = distance(next_state, goal)
    action = np.random.choice(np.flatnonzero(action_scores == action_scores.min()))
    return torch.tensor([[action]], device=device, dtype=torch.long)    

def sample_expert_trajectory():
    state = start_state
    next_state = None

    trajectory = []
    
    for t in range(EPISODE_LENGTH):
        action = select_expert_action(state, end_state)
        
        torch_state = torch.from_numpy(state).float().unsqueeze(0)
        next_state = step(state, action.item())
        torch_next_state = torch.from_numpy(next_state).float().unsqueeze(0)
        reward = get_reward(state, action.item(), next_state)
        done = is_done(state)
        reward = torch.tensor([reward], device=device)
        if done:
            torch_next_state = None

        trajectory.append((torch_state, action, torch_next_state, reward))
        if done:
            break
        state = next_state
    return trajectory

In [87]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,batch.next_state)),
                                  device=device, dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    #DDQN
    policy_action_indices = policy_net(non_final_next_states).max(1)[1].detach()
    target_actions = target_net(non_final_next_states).detach()
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_actions.gather(1, policy_action_indices.unsqueeze(1)).squeeze()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    return loss.item()

In [88]:
losses = [0]
torch_end = torch.from_numpy(end_state).float().unsqueeze(0)
for i in range(1000):
    expert_trajectory = sample_expert_trajectory()
    for torch_state, action, torch_next_state, reward in expert_trajectory:
        memory.push(combine(torch_state, torch_end), action, combine(torch_next_state, torch_end), reward)

for i_episode in range(num_episodes):
    next_state = None
    final = i_episode == num_episodes - 1
    trajectory = []
    
    if i_episode % 500 == 0:
        scheduler.step()
    if BACKPLAY:
        start_index = np.random.choice(len(expert_trajectory))
        if np.random.random_sample() < 0.2 or final:
            start_index = 0
        state = expert_trajectory[start_index][0].squeeze().data.numpy()
    else:
        state = start_state
    
    
    
    for t in range(EPISODE_LENGTH):
        torch_state = torch.from_numpy(state).float().unsqueeze(0)

        action = select_action(torch_state, torch_end, final)
            
        next_state = step(state, action.item())
        torch_next_state = torch.from_numpy(next_state).float().unsqueeze(0)
        reward = get_reward(state, action.item(), next_state)
        done = is_done(state)
        reward = torch.tensor([reward], device=device)

        if done:
            torch_next_state = None

        trajectory.append((torch_state, action, torch_next_state, reward))

        loss = optimize_model()
        losses.append(loss if loss is not None else 0)
        if final:
            print(state)
        if done:
            break
        state = next_state

    if i_episode % 1000 == 0:
        print(i_episode)
        print(np.mean(np.asarray(losses)))
        losses = []

    #HER updates with goal as real goal OR last state in trajectory
    for torch_state, action, torch_next_state, reward in trajectory:
        memory.push(combine(torch_state, torch_end), action, combine(torch_next_state, torch_end), reward)
        if not is_done(state):
            torch_local_goal = torch.from_numpy(state).float().unsqueeze(0)
            local_reward = torch.equal(torch_next_state, torch_local_goal)
            new_reward = torch.add(reward, local_reward)
            memory.push(combine(torch_state, torch_local_goal), action, combine(torch_next_state, torch_local_goal), new_reward)
                    
                          
    # Update the target network
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    
        
print('Complete')




0
0.11249264578024547
1000
0.12283444515271616
2000
0.06946847996752221
3000
0.07281742867984065
4000
0.05974162167285024
5000
0.019982431982169434
6000
0.007087836938796034
7000
0.006390875907770631
8000
0.005219049782663524
9000
0.003941521353056366
10000
0.0035706456431785507
11000
0.002784720499500576
12000
0.001911910399530583
13000
0.00143279895960903
14000
0.0012930351541785103
15000
0.0015171439998489683
16000
0.0017227342542957282
17000
0.0017810532811767014
18000
0.00196029058123022
19000
0.0018758286524774542
[0. 0. 0. 0.]
[0. 1. 0. 0.]
[0. 2. 0. 0.]
[0. 3. 1. 0.]
[1. 3. 1. 0.]
[1. 2. 1. 0.]
[1. 1. 1. 0.]
[1. 0. 1. 0.]
[2. 0. 1. 0.]
[3. 0. 1. 1.]
[4. 0. 1. 1.]
[4. 1. 1. 1.]
[4. 2. 1. 1.]
[4. 3. 1. 1.]
[4. 4. 1. 1.]
Complete


In [94]:
def predicted_distance(state, goal):
    torch_state = torch.from_numpy(state).float().unsqueeze(0)
    torch_goal = torch.from_numpy(goal).float().unsqueeze(0)
    x = torch.cat((torch_state, torch_goal), 1)
    return -1 * policy_net(x).max(1)[0].view(1, 1).item()

def predicted_measure_ball(state, goal, n = 500):
    measure = {}
    torch_state = torch.from_numpy(state).float().unsqueeze(0)
    torch_goal = torch.from_numpy(goal).float().unsqueeze(0)
    x = torch.cat((torch_state, torch_goal), 1)
    for i in range(n):
        action = select_action_entropy(torch_state, torch_goal, alpha = 4.0)
        next_state = step(state, action.item())
        key = tuple(next_state)
        measure[key] = measure.get(key, 0) + 1.0/n
    return measure

def wasserstein(a, b, goal):
    if tuple(a) == tuple(b):
        return 0
    m_a = predicted_measure_ball(a, goal)
    m_b = predicted_measure_ball(b, goal)
    states = list(set(m_a.keys()) | set(m_b.keys()))
    
    m_a = [m_a.get(s, 0.0) for s in states]
    m_b = [m_b.get(s, 0.0) for s in states]
    dist = [[distance(np.asarray(s1),np.asarray(s2)) for s1 in states] for s2 in states]
    return emd(np.asarray(m_a), np.asarray(m_b), np.asarray(dist))

def curvature(a, b, goal):
    return 1 - wasserstein(a, b, goal) / max(distance(a, b), 0.001)

def local_curvature(state, goal):
    m_s = predicted_measure_ball(state, goal)
    return sum([m_s[key] * curvature(state, np.asarray(key), goal) for key in m_s.keys()])

def print_space(fn, goal):
    np.set_printoptions(precision=2, suppress=True)
    for red in [0,1]:
        for blue in [0,1]:
            grid = np.zeros((num_row, num_col))
            for row in range(num_row):
                for col in range(num_col):
                    state = np.asarray((row, col, blue, red))
                    grid[row, col] = fn(state, goal)
            print("red = " + str(red))
            print("blue = " + str(blue))
            print(grid)

a = np.asarray([2,0,0,0])
b = np.asarray([1,0,0,0])
print(predicted_measure_ball(start_state, end_state))
print(curvature(a,b,end_state))

{(0, 0, 0, 0): 0.022000000000000006, (1, 0, 0, 0): 0.44800000000000034, (0, 1, 0, 0): 0.5300000000000004}
0.008000973999999217


In [95]:
print_space(distance, end_state)
print('\n')
print_space(predicted_distance, end_state)

red = 0
blue = 0
[[14. 13. 12. 11. 12.]
 [13. 14. 13. 12. 13.]
 [12. 13. 14. 13. 14.]
 [11. 12. 13. 14. 15.]
 [12. 13. 14. 15. 16.]]
red = 0
blue = 1
[[ 8.  9. 10. 11. 12.]
 [ 7.  8.  9. 10. 11.]
 [ 6.  7.  8.  9. 10.]
 [ 5.  6.  7.  8.  9.]
 [ 6.  7.  8.  9. 10.]]
red = 1
blue = 0
[[ 8.  7.  6.  5.  6.]
 [ 9.  8.  7.  6.  7.]
 [10.  9.  8.  7.  8.]
 [11. 10.  9.  8.  9.]
 [12. 11. 10.  9. 10.]]
red = 1
blue = 1
[[8. 7. 6. 5. 4.]
 [7. 6. 5. 4. 3.]
 [6. 5. 4. 3. 2.]
 [5. 4. 3. 2. 1.]
 [4. 3. 2. 1. 0.]]


red = 0
blue = 0
[[13.84 12.92 11.99 12.31 14.74]
 [12.87 13.67 13.02 14.33 15.66]
 [11.88 12.62 14.31 15.26 15.4 ]
 [10.68 12.58 14.14 14.77 13.89]
 [10.71 12.41 13.16 13.56 12.79]]
red = 0
blue = 1
[[ 7.96  8.96  9.98 10.96 11.87]
 [ 6.95  7.97  8.99  9.98 10.97]
 [ 5.93  6.97  8.01  9.04 10.04]
 [ 6.44  7.    7.76  8.67  9.62]
 [ 7.38  7.58  8.01  8.92  9.03]]
red = 1
blue = 0
[[ 8.26  6.91  5.92  6.2   5.73]
 [ 8.9   7.94  6.91  5.97  6.79]
 [ 9.9   8.94  7.91  7.22  6.48]
 [10.91 1

In [96]:
print_space(local_curvature, end_state)

red = 0
blue = 0
[[-0.93 -0.01  0.02  0.78  0.23]
 [ 0.01 -0.44 -0.28 -0.47  0.28]
 [ 0.01 -0.03 -0.07 -0.04 -0.11]
 [ 0.49 -0.4  -0.   -0.02 -0.33]
 [ 0.21  0.21 -0.   -0.14 -0.25]]
red = 0
blue = 1
[[ 0.03  0.01  0.   -0.02  0.01]
 [ 0.06 -0.03 -0.   -0.   -0.09]
 [-0.01  0.01 -0.02 -0.01 -0.03]
 [ 0.95 -0.34  0.04 -0.    0.02]
 [ 0.93  0.71  0.35  0.22  0.12]]
red = 1
blue = 0
[[ 0.03  0.08 -0.02  0.5   0.  ]
 [ 0.    0.02  0.06 -0.04  0.  ]
 [-0.   -0.    0.   -0.    0.  ]
 [ 0.01 -0.01  0.    0.    0.  ]
 [ 0.01  0.    0.   -0.   -0.  ]]
red = 1
blue = 1
[[-0.23 -0.18 -0.23 -0.02  0.17]
 [-0.28 -0.1  -0.26 -0.07  0.08]
 [-0.01 -0.02 -0.01  0.01  0.01]
 [-0.14  0.    0.    0.02  0.53]
 [ 0.03  0.02  0.04  0.57  0.78]]
